### Example 10: Poisson / Negative Binomial Rate Change

### 2.1 What to do?
In this example we compare event **counts** across multiple time-points using
`statsmed.poisson_negbin_rate_change()`.

The function requires only two arguments beyond the DataFrame: `time_col` and `count_col`.
Everything else is optional:
- **`timepoints`** – auto-detected (sorted unique values) when omitted.
- **`id_col`** – set to `None` when there is only one unit (no clustering needed).
- **`exposure_col`** – omit when there is no offset / exposure.

Steps:
1. Load `ex10.csv` (80 units, 4 years, readmission counts + discharges).
2. **Minimal call** – just `time_col` and `count_col`.
3. **Categorical** – RR for each year vs. reference (2019), with clustering and exposure.
4. **Trend** – single per-year RR (linear trend).
5. **Two-timepoint subset** – `timepoints=[2019, 2022]`.
6. **Single unit** (no `id_col`) – aggregated data.
7. **Overdispersion check** + Negative Binomial.
8. **`report_rr()`** as standalone helper.

In [ ]:
import pandas as pd
import numpy as np
from statsmed import statsmed

# Load data
df = pd.read_csv('ex10.csv')
print(df.head(12))
print(f"\nShape: {df.shape}")
print(f"Unique units: {df['unit_id'].nunique()}")
print(f"Years: {sorted(df['year'].unique().tolist())}")

In [ ]:
# --- Step 1: Minimal call -- only time_col and count_col ---
# timepoints, id_col, exposure_col are all auto / None
rr_min, fit_min = statsmed.poisson_negbin_rate_change(
    df, time_col='year', count_col='readmissions',
)
print('\nReturned dict:')
for k, v in rr_min.items():
    print(f'  {k}: {v}')

In [ ]:
# --- Step 2: Categorical with id_col + exposure ---
rr_cat, fit_cat = statsmed.poisson_negbin_rate_change(
    df,
    time_col='year',
    count_col='readmissions',
    id_col='unit_id',
    exposure_col='discharges',
)
print('\nReturned keys:', list(rr_cat.keys()))

In [ ]:
# --- Step 3: Trend ---
rr_trend, fit_trend = statsmed.poisson_negbin_rate_change(
    df,
    time_col='year',
    count_col='readmissions',
    id_col='unit_id',
    time_as='trend',
    exposure_col='discharges',
)
print('\nRR per year:', rr_trend['RR'])

In [ ]:
# --- Step 4: Two-timepoint subset (2019 vs 2022 only) ---
rr_2tp, fit_2tp = statsmed.poisson_negbin_rate_change(
    df,
    time_col='year',
    count_col='readmissions',
    timepoints=[2019, 2022],
    exposure_col='discharges',
)

In [ ]:
# --- Step 5: Single unit (no id_col) ---
# Aggregate to one row per year (e.g. one device, one hospital)
df_agg = df.groupby('year').agg(
    readmissions=('readmissions', 'sum'),
    discharges=('discharges', 'sum'),
).reset_index()
print(df_agg)
print()

rr_single, fit_single = statsmed.poisson_negbin_rate_change(
    df_agg,
    time_col='year',
    count_col='readmissions',
    exposure_col='discharges',   # id_col omitted -> no clustering
)

In [ ]:
# --- Step 6: Overdispersion check ---
disp = statsmed.quick_overdispersion_check_poisson(fit_cat)
print(f"Pearson chi2 / df_resid = {disp:.3f}")
if disp > 1.5:
    print("  -> Overdispersion detected. Consider Negative Binomial.")
else:
    print("  -> No strong overdispersion. Poisson model is adequate.")

In [ ]:
# --- Step 7: Negative Binomial ---
rr_nb, fit_nb = statsmed.poisson_negbin_rate_change(
    df,
    time_col='year',
    count_col='readmissions',
    id_col='unit_id',
    exposure_col='discharges',
    model='negbin',
)

In [ ]:
# --- Step 8: report_rr() standalone ---
rr_manual = statsmed.report_rr(fit_trend, '_time_num', N_of_decimals=4)
print('report_rr() output:', rr_manual)

We compared readmission rates across four years using Poisson and Negative Binomial regression.

- **Categorical** analysis showed decreasing Rate Ratios for each year relative to 2019.
- **Trend** analysis yielded a single per-year RR summarizing the monotonic decline.
- A **two-timepoint** comparison is achieved by passing only two elements in `timepoints`.
- When there is only **one unit**, simply omit `id_col` (no clustering/FE).
- `report_rr()` can be used standalone on any statsmodels regression result.

### 2.2 What to write?

- In Methods (categorical): "We modeled the change in readmission counts across four years
  using Poisson regression with treatment-coded dummy variables (reference: 2019), a log-offset
  for the number of discharged patients, and cluster-robust standard errors at the unit level.
  Exponentiated coefficients were interpreted as Rate Ratios (RR) with 95% Wald confidence
  intervals."
- In Methods (trend): "To test for a monotonic trend, we entered year as an ordinal predictor
  (0, 1, 2, 3) in a Poisson log-linear model. The exponentiated coefficient represents the
  multiplicative change in rate per year."
- Sensitivity: "As a sensitivity analysis, we repeated the analysis using a Negative Binomial
  (NB2) model to account for potential overdispersion."
- Results: "Readmission rates declined over the study period (trend RR = X.XX per year,
  95% CI X.XX–X.XX; p = X.XXX). Compared to 2019, rates in 2022 were reduced by approximately
  XX% (RR = X.XX, 95% CI X.XX–X.XX; p = X.XXX)."